# 2.1 理解词嵌入(enbedding)

深度神经网络模型，包括大型语言模型（LLMs），是无法直接处理原始文本。
由于文本属于分类数据，它与用于实施和训练神经网络的数学运算不兼容。
因此，我们需要一种方法将单词表示为连续值向量。 
（不熟悉在上下文中进行向量和张量计算的读者可以在附录A，A2.2《理解张量》中了解更多信息。）

将数据转换为向量格式的概念通常被称为嵌入(embedding)。
如图2.2所示，通过使用特定的神经网络层或其他预训练的神经网络模型，我们可以将不同类型的数据嵌入，例如视频、音频和文本。

**图2.2 
深度学习模型无法处理视频、音频和文本等原始数据格式。
因此，我们使用嵌入模型将这些原始数据转换为密集向量表示，这样深度学习架构就可以轻松理解和处理原始数据。
具体来说，此图展示了将原始数据转换为三维数值向量的过程。
需要注意的是，不同的数据格式需要不同的嵌入模型。
例如，为文本设计的嵌入模型不适用于嵌入音频或视频数据。**

![fig2.2](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-2-2.jpg?raw=true)

从本质上讲，嵌入是一种映射，它将离散对象，如单词、图像甚至整个文档，映射到连续向量空间中的点——嵌入的主要目的是将非数字数据转换为神经网络可以处理的格式。

虽然词嵌入是文本嵌入最常见的形式，但也可以用于句子、段落或整个文档的嵌入。
句子或段落嵌入是检索增强生成的流行选择。检索增强生成结合了生成（如产生文本）和检索（如搜索外部知识库）两种方式，以在生成文本时拉取相关信息，这些技术超出了本书讲述的范围。
由于我们的目标是训练类似GPT的大型语言模型（LLMs），这些模型学习一次生成一个单词的文本，因此本章着重介绍词嵌入。

目前已经开发了多种算法和框架来生成词嵌入。
其中最早和最流行的例子之一是 Word2Vec 方法。
Word2Vec 训练神经网络架构是通过预测给定目标词的上下文或反之来生成词嵌入。
Word2Vec 架构的主要思想是，出现在相似上下文中的词往往具有相似的含义。
因此，当将词嵌入投影到二维空间中以便于可视化时，可以看到相似的术语聚集在一起，如图2.3所示。

**图2.3 如果词嵌入是二维的，我们可以将它们绘制在二维散点图中以便于可视化，如此图所示。在使用词嵌入技术，例如 Word2Vec 时，对应于相似概念的词在嵌入空间中通常彼此接近。例如，在嵌入空间中，不同类型的鸟类相对于国家和城市更为靠近。**

![fig2.3](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-2-3.jpg?raw=true)

词嵌入可以有不同的维度，从一维到数千维不等。
如图2.3所示，我们可以选择二维词嵌入以便于可视化。
更高的维度可能会捕捉到词之间更多细微的关系，但作为代价，计算效率将会下降。

虽然我们可以使用预训练模型如 Word2Vec 来为机器学习模型生成嵌入，但大型语言模型（LLMs）通常会生成它们自己的嵌入，这些嵌入是输入层的一部分，并在训练期间更新。
将嵌入作为 LLM 训练的一部分进行优化，而不是使用 Word2Vec 的优势在于，嵌入被优化以适应手头的特定任务和数据。
我们将在本章后面部分实现这样的嵌入层。
此外，正如我们在第3章中讨论的，LLMs 还可以创建上下文化的输出嵌入。

不幸的是，高维嵌入在可视化上存在挑战，因为我们的感官感知和常见的图形表示本质上限于三维或更少，这就是为什么图2.3展示了在二维散点图中的二维嵌入)的原因。
然而，在处理大型语言模型（LLMs）时，我们通常使用的嵌入维度远高于图2.3中显示的维度。
对于GPT-2和GPT-3，嵌入大小（通常被称为模型隐藏状态的维度）根据具体的模型变种和大小而变化。
这是性能与效率之间的权衡。
最小的GPT-2（1.17亿参数）和GPT-3（1.25亿参数）模型使用768维的嵌入大小来提供具体示例。最大的GPT-3模型（1750亿参数）使用的嵌入大小为12,288维。

本章接下来的部分将介绍为大型语言模型（LLM）准备嵌入所需的那些步骤，包括将文本分割成单词、将单词转换为令牌，以及将令牌转换为嵌入向量。